# Experimento y Evaluación de Desempeño:

## a) Búsqueda en texto:
Evalúe el rendimiento de su implementación del índice invertido comparándolo con los
resultados obtenidos utilizando PostgreSQL. Para ello:
- Diseñe consultas equivalentes en ambos sistemas y mida el tiempo de respuesta y la calidad
de los resultados (¿Se obtiene los mismo resultados?).
- Investigue y explique cómo PostgreSQL realiza la recuperación de información textual:
    - El tipo de índice que utiliza (por ejemplo, `GIN` o `GiST`).
    - La función de similitud o ranking que emplea (`ts_rank`, `ts_rank_cd`, entre otras).
    - Cómo gestiona el procesamiento de consultas en campos tsvector y tsquery.

### 1. Diseño de consultas equivalentes

Se evaluó la búsqueda de texto utilizando la consulta "trump obama republican politics democrat" sobre una colección de noticias, aplicando tanto el índice invertido `SPIMI` en la base de datos propia como el índice `GIN` sobre tsvector en PostgreSQL.
En ambos sistemas, se recuperaron los 5 resultados más relevantes (`k=5`) para comparar eficiencia y calidad de respuesta.

### Descripción del Dataset

Se utilizó el **Fake and Real News Dataset** de Kaggle, que contiene miles de noticias clasificadas como verdaderas o falsas.  
Para este experimento, se combinaron ambos archivos (`True.csv` y `Fake.csv`) y se seleccionaron los campos **`id`** y **`title`** para simplificar las pruebas de búsqueda textual.

El procesamiento del CSV se realizó de la siguiente manera:
- Se cargaron ambos archivos usando `pandas.read_csv`.
- Se concatenaron los dos DataFrames con `pd.concat`.
- Se asignó un **id único** a cada registro usando `df["id"] = range(1, len(df) + 1)`.
- Se seleccionaron solo las columnas relevantes: **`id`** y **`title`**.
- El resultado se truncó a diferentes tamaños (`size`), variando desde **10** hasta **32,000** registros para evaluar el rendimiento y la escalabilidad de las búsquedas.
- Finalmente, el dataset procesado se guardó en un nuevo archivo CSV: **`news_large.csv`**.

#### Consulta en nuestra DB:

In [ ]:
from database import search_text
table_name = "news_text"
search_text(table_name, "trump obama republican politics democrat", 5)

#### Consulta en PostgreSQL:

In [ ]:
SELECT title
FROM news_text
ORDER BY ts_rank(title_tsv, plainto_tsquery('english', 'trump obama republican politics democrat')) DESC
LIMIT 5;

### 2. Medición de tiempo de respuesta y calidad de resultados

Se midió el tiempo de inserción y búsqueda para distintos tamaños de la colección (desde 10 hasta 32,000 registros).
Los resultados se guardaron en performance_results_text.csv y se visualizaron en el notebook.

#### Tabla de mediciones de tiempo:

In [6]:
import pandas as pd
df = pd.read_csv("performance_results_text.csv")
display(df)

,size,creation_db,creation_pg,search_db,search_pg
0,10,309.219187,0.012449,1.242528,0.010138
1,100,29.037008,0.014726,0.070118,0.009375
2,1000,320.622391,0.050767,1.292852,0.011924
3,2000,630.946300,0.073056,4.747565,0.010663
4,4000,1239.925486,0.138207,18.292634,0.011962
5,8000,2460.187505,0.253865,78.445359,0.015688
6,16000,5248.943822,0.508398,222.031922,0.019747
7,32000,14337.485067,1.247853,827.569731,0.033326


#### Resultados de búsqueda:
Nuestra implementacion:
- Senior U.S. Republican senator: 'Let Mr. Mueller do his job'
- As U.S. budget fight looms, Republicans flip their fiscal script
- Trump on Twitter (Dec 28) - Global Warming
- Factbox: Trump on Twitter (Dec 29) - Approval rating, Amazon
- Trump says Russia probe will be fair, but timeline unclear: NYT

PostgreSQL:
- As U.S. budget fight looms, Republicans flip their fiscal script
- U.S. military to accept transgender recruits on Monday: Pentagon
- Senior U.S. Republican senator: 'Let Mr. Mueller do his job'
- FBI Russia probe helped by Australian diplomat tip-off: NYT
- Trump wants Postal Service to charge 'much more' for Amazon shipments

#### Observaciones:
- **Eficiencia**:
PostgreSQL demostró ser mucho más eficiente tanto en la inserción como en la búsqueda de texto. Los tiempos de respuesta fueron significativamente menores en comparación con la implementación propia basada en índice invertido `SPIMI`.
- **Calidad de resultados**:
La relevancia de los títulos recuperados fue similar en ambos sistemas para las consultas realizadas. Sin embargo, el orden de los resultados puede variar debido a las diferencias en los algoritmos de ranking utilizados.
- **Escalabilidad**:
La implementación propia funciona correctamente para volúmenes pequeños y medianos de datos, pero su tiempo de inserción y búsqueda aumenta considerablemente con el tamaño de la colección, lo que limita su uso en escenarios de grandes volúmenes.

### 3. Investigación sobre recuperación de información textual en PostgreSQL

#### a) Tipo de índice utilizado
PostgreSQL utiliza principalmente dos tipos de índices para búsquedas de texto completo en columnas de tipo tsvector:
- PostgreSQL utiliza principalmente `GIN` (Generalized Inverted Index) para búsquedas de texto completo en columnas tsvector.
- También puede usar `GiST` (Generalized Search Tree), aunque `GIN` es más eficiente para la mayoría de consultas de texto.

#### b) Función de similitud o ranking
Las funciones principales son `ts_rank` y `ts_rank_cd`
- **ts_rank**: Calcula la relevancia considerando frecuencia y posición de los términos.
- **ts_rank_cd**: Usa el algoritmo _"cover density"_ para ranking alternativo.

### Conclusión
- PostgreSQL supera ampliamente en eficiencia y escalabilidad a la implementación propia.
- La calidad de los resultados es comparable, aunque el ranking puede variar.
- El uso de índices `GIN` y funciones de ranking como `ts_rank` hacen que PostgreSQL sea ideal para búsquedas de texto completo en grandes volúmenes de datos.